<a href="https://colab.research.google.com/github/sayakpaul/Generalized-ODIN-TF/blob/main/Calculate_Epsilon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup

In [3]:
# Retrieve the pre-trained ResNet20 model.
!gdown --id 1-6Xp7__VMJeMBtGMByI7_GYVeIIv-1Ay -O odin_rn_model.tar.gz
!tar xf odin_rn_model.tar.gz

Downloading...
From: https://drive.google.com/uc?id=1-6Xp7__VMJeMBtGMByI7_GYVeIIv-1Ay
To: /content/odin_rn_model.tar.gz
6.64MB [00:00, 213MB/s]


In [1]:
import tensorflow as tf

import matplotlib.pyplot as plt
import numpy as np

tf.random.set_seed(42)
np.random.seed(42)

## Load the pre-trained model

In [2]:
model = tf.keras.models.load_model("odin_rn_model")
print(f"Pre-trained model loaded with {model.count_params()/1e6} M parameters.")

Pre-trained model loaded with 0.574572 M parameters.


## Load the CIFAR-10 dataset

In [3]:
(_, _), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
print(f"Total test examples: {len(x_test)}")

Total test examples: 10000


## Define constants

In [4]:
BATCH_SIZE = 128
AUTO = tf.data.AUTOTUNE
SAMPLES_PERTURB = 1000

## Prepare data loaders

In [5]:
perturb_ds = tf.convert_to_tensor(x_test[:SAMPLES_PERTURB].astype("float32"))
ds = tf.data.Dataset.from_tensor_slices((x_test, y_test))
test_ds = ds.skip(SAMPLES_PERTURB).take(len(x_test) - SAMPLES_PERTURB).batch(BATCH_SIZE)

## Calculating the perturbation magnitude ($\epsilon^{*}$)

From the paper: 

> In our method, we search for the $\epsilon^{*}$ which maximizes the score $S(x)$ with only the in-distribution validation dataset $D_{\text {in }}^{\text {val }}$:

$$\epsilon^{*}=\underset{\varepsilon}{\arg \max } \sum_{x \in D_{i n}^{\text {val }}} S({\boldsymbol{x}})$$

$S(x)$ is given by:

>  For out-of-distribution detection, we use the scoring function:
<center>
$S_{DeConf}(\boldsymbol{x})=\max _{i} h_{i}(\boldsymbol{x})$ or $g(\boldsymbol{x})$
</center>

Perturbation of an input image is realized using the equation below:

$$
\hat{\boldsymbol{x}}=\boldsymbol{x}-\epsilon \operatorname{sign}\left(-\nabla_{\boldsymbol{x}} S(\boldsymbol{x})\right)
$$

In [6]:
# Let's define our model to obtain scores.
scorer = tf.keras.Model(model.input, model.layers[-2].output)

In [7]:
# We can take a batch of data and obtain the OOD scores like so.
sample_scores = scorer.predict(test_ds.take(1))
sample_scores[:5]

array([[0.4178029 , 0.2997888 , 0.42754278, 0.2922473 , 0.63637024,
        0.33219904, 0.2604284 , 0.44775465, 0.25829434, 0.4107703 ],
       [0.19263816, 0.34350842, 0.3622666 , 0.35726658, 0.4652104 ,
        0.4666784 , 0.29159823, 0.32265878, 0.17091236, 0.30805007],
       [0.371202  , 0.41554976, 0.502523  , 0.4851712 , 0.6317208 ,
        0.42792296, 0.35079482, 0.33695242, 0.22708139, 0.35269815],
       [0.49741137, 0.34551892, 0.5209366 , 0.50705826, 0.3402585 ,
        0.38563576, 0.3555499 , 0.38100013, 0.3735423 , 0.61080885],
       [0.63022316, 0.6320237 , 0.727445  , 0.73132706, 0.902592  ,
        0.80281264, 0.61695886, 0.7478932 , 0.29849046, 0.8099188 ]],
      dtype=float32)

In [8]:
# Grid as defined in Section 3.2.
epsilon_grid = [0.0025, 0.005, 0.01, 0.02, 0.04, 0.08]

In [11]:
def perturb_images(model, epsilon):
    test_ds_var = tf.Variable(perturb_ds, trainable=True)
    
    with tf.GradientTape() as tape:
        # Calculate the scores.
        tape.watch(test_ds_var)
        scores = model(test_ds_var, training=False)

    # Calculate the gradients of the scores with respect to the inputs.
    # (Notice the sign.)
    gradients = -tape.gradient(scores, test_ds_var)

    # Perturb the inputs and derive new mean score.
    test_ds_var.assign_sub(epsilon * tf.sign(gradients))
    static_tensor = tf.convert_to_tensor(test_ds_var)
    new_mean_score = tf.reduce_mean(model.predict(static_tensor))

    return new_mean_score

In [14]:
# Derive the perturbation magnitude. 
mean_scores = []

for epsilon in epsilon_grid:
    mean_scores.append(perturb_images(scorer, epsilon))

epsilon_selected = epsilon_grid[np.argmax(mean_scores)]
print(f"Epsilon: {epsilon_selected}")

Epsilon: 0.08
